In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
from datetime import datetime
from datetime import timedelta

In [3]:
import numpy as np
import pandas as pd

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [5]:
data = pd.read_csv(r'C:\Users\Gleb\Desktop\\retail.csv')
data = data.drop(['Unnamed: 0'], axis=1)

In [7]:
others = data.loc[data['store_name'] != 'Райффайзенбанк']
others = others.drop(['mcc'], axis=1)
others = others.reset_index(drop=True)
others.purchdate = pd.to_datetime(others.purchdate)

In [8]:
test_others = others.copy()
test_others['days_norm'] = 0
test_others.loc[test_others['purchdate'].dt.month == 9, 
                              'days_norm'] = test_others.purchdate.dt.day
test_others.loc[test_others.purchdate.dt.month == 10, 
                              'days_norm'] = test_others.purchdate.dt.day + 30
test_others.loc[test_others.purchdate.dt.month == 11, 
                              'days_norm'] = test_others.purchdate.dt.day + 61
test_others.loc[test_others.purchdate.dt.month == 12, 
                              'days_norm'] = test_others.purchdate.dt.day + 91
test_others.days_norm -= 16

test_others['sets'] = test_others.days_norm // 7 + 1
test_others = test_others[test_others.sets != 14]

In [9]:
others_final = pd.DataFrame()
others_final['cnum'] = others.cnum.value_counts().index

,cnum
0,E46NJV
1,0HGSDX
2,JMQTML
3,JKKNTP
4,E4BVRA
...,...
49356,8EEVLJ
49357,IKPJJJ
49358,4YDCXF
49359,E45DZO


In [10]:
for i in range(1, 14):
    tmp = pd.DataFrame(test_others.loc[test_others['sets'] == i]).groupby('cnum').agg(
    com_ava_2 = ('amount', lambda x: sum(x) / len(x)),
    com_freq_2 = ('amount', len))
    
    others_final = pd.merge(others_final, tmp, on='cnum', how='left')

In [12]:
others_final.columns = ['cnum', 'oth_av_1', 'oth_freq_1', 'oth_av_2', 'oth_freq_2', 'oth_av_3', 'oth_freq_3', 'oth_av_4',
                       'oth_freq_4', 'oth_av_5', 'oth_freq_5', 'oth_av_6', 'oth_freq_6', 
                        'oth_av_7', 'oth_freq_7', 'oth_av_8',
                        
                        'oth_freq_8', 'oth_av_9', 'oth_freq_9', 'oth_av_10', 'oth_freq_10',
                       'oth_av_11', 'oth_freq_11', 'oth_av_12', 'oth_freq_12', 'oth_av_13', 
                        'oth_freq_13'
                       ]
others_final['Summa'] = others_final[others_final.columns[range(2, 28, 2)]].sum(axis=1)

## Clients

In [13]:
clients = data.loc[data['store_name'] == 'Райффайзенбанк']
clients = clients.drop(['mcc'], axis=1)
clients = clients.reset_index(drop=True)
clients.purchdate = pd.to_datetime(clients.purchdate)

In [14]:
test_clients = clients.copy()
test_clients['days_norm'] = 0
test_clients.loc[test_clients['purchdate'].dt.month == 9, 
                              'days_norm'] = test_clients.purchdate.dt.day
test_clients.loc[test_clients.purchdate.dt.month == 10, 
                              'days_norm'] = test_clients.purchdate.dt.day + 30
test_clients.loc[test_clients.purchdate.dt.month == 11, 
                              'days_norm'] = test_clients.purchdate.dt.day + 61
test_clients.loc[test_clients.purchdate.dt.month == 12, 
                              'days_norm'] = test_clients.purchdate.dt.day + 91
test_clients.days_norm -= 18
test_clients['sets'] = test_clients.days_norm // 7 + 1
test_clients = test_clients[test_clients.sets != 14]

In [15]:
clients_final = pd.DataFrame()
clients_final['cnum'] = clients.cnum.value_counts().index

,cnum
0,EL80P2
1,RIPTMP
2,NJMQPR
3,7HSYSN
4,JLQMIT
...,...
260702,EEPHFX
260703,E459MS
260704,ELTVVF
260705,MLLTLN


In [16]:
for i in range(1, 14):
    tmp = pd.DataFrame(test_clients.loc[test_clients['sets'] == i]).groupby('cnum').agg(
    com_ava_2 = ('amount', lambda x: sum(x) / len(x)),
    com_freq_2 = ('amount', len))
    
    clients_final = pd.merge(clients_final, tmp, on='cnum', how='left')

In [18]:
clients_final.columns = ['cnum', 'our_av_1', 'our_freq_1', 'our_av_2', 'our_freq_2', 'our_av_3', 'our_freq_3', 'our_av_4',
                       'our_freq_4', 'our_av_5', 'our_freq_5', 'our_av_6', 'our_freq_6', 
                        'our_av_7', 'our_freq_7', 'our_av_8',
                        
                        'our_freq_8', 'our_av_9', 'our_freq_9', 'our_av_10', 'our_freq_10',
                       'our_av_11', 'our_freq_11', 'our_av_12', 'our_freq_12', 'our_av_13', 
                        'our_freq_13'
                       ]
clients_final['Summa'] = clients_final[clients_final.columns[range(2, 28, 2)]].sum(axis=1)

In [20]:
all_final = clients_final.merge(others_final, on='cnum', how='inner')
del clients
del others
all_final = all_final.drop(['Summa_y', 'Summa_x'], axis=1)


In [21]:
tmp = clients_final[clients_final.columns[2:28:2]]
tmp['cnum'] = clients_final.cnum
tmp[tmp.isna().sum(axis=1) < 10]
tmp.index = tmp.cnum
tmp.drop(['cnum'], axis=1, inplace=True)

i = 0
to_explore = []
while i + 3 <= tmp.shape[1]:
    y = tmp[tmp.columns[i:i+3]].isna().sum(axis=1)
    to_explore.append(y[y == 0].index.values)
    i += 1

In [22]:
empty = pd.DataFrame(columns=['1st_week_aver', '1st_week_freq', '2nd_week_aver', '2nd_week_freq', 
                             '3rd_week_aver', '3rd_week_freq', 'cnum', 'batch_num', 'target'])
i = 0
while i < len(to_explore):
    y = clients_final[clients_final.cnum.isin(to_explore[i])]
    j = 2*i + 1
    y2 = y[y.columns[j:j+6]]
    y2['cnum'] = y.cnum
    y2['batch_num'] = i + 1
    if j+7 < 28: 
        y2['target'] = y[y.columns[j+7]].isna()
    else:
        y2['target'] = np.nan
    y2.columns = empty.columns
    empty = empty.append(y2)
    i += 1
    
empty.target = empty.target.replace({True:0, False:1})

# Model

In [52]:
train = empty.drop(['cnum'], axis=1)
train = train[train.target.isna() == False]
x_train = train.drop(['target'], axis=1)
y_train = train.target

test = empty.drop(['cnum'], axis=1)
test = test[test.target.isna()]
test = test.drop(['target'], axis=1)

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn import metrics

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_train, y_train, random_state=42)

clf2 = RandomForestClassifier()
clf2.fit(X_train, Y_train)
prediction = clf2.predict(X_test)
metrics.accuracy_score(Y_test, prediction)

0.6053603857473856

# Continue

In [24]:
leaved = empty[empty.target == 0]
leaved_others = others_final[others_final.cnum.isin(leaved.cnum)]
leaved_others = leaved_others[leaved_others.columns[0:28:2]]
leaved_others.index = leaved_others.cnum
leaved_others = leaved_others.drop(['cnum'], axis=1)

In [31]:
# True -- не пошел к конкуренту
# False -- пошел к конкуренту
compet_or_not = pd.DataFrame(columns=['cnum', 'batch_num', 'to_compet'])
for i in range(1, 11):
    tmp = leaved[leaved.batch_num == i]
    tmp_others = leaved_others[leaved_others.index.isin(tmp.cnum)]
    not_compet = tmp_others[tmp_others.columns[i + 2]].isna()
    to_append = pd.DataFrame({'cnum': not_compet.index.values, 'batch_num': i, 
                                          'to_compet': not_compet.values})
    compet_or_not = compet_or_not.append(to_append)
empty = empty.merge(compet_or_not, how='left', on=['batch_num', 'cnum'])
empty.loc[empty.to_compet == False, 'target'] = 2
empty = empty.drop(['to_compet'], axis=1)

# Continue x2

In [56]:
value_lost = empty[['cnum', 'batch_num', 'target']]
value_lost = value_lost.dropna()

In [64]:
x = others_final[others_final.columns[1:27:2]]
x['cnum'] = others_final.cnum


,oth_av_1,oth_av_2,oth_av_3,oth_av_4,oth_av_5,oth_av_6,oth_av_7,oth_av_8,oth_av_9,oth_av_10,oth_av_11,oth_av_12,oth_av_13,cnum
0,NaN,NaN,NaN,NaN,10791.666667,24985.714286,31181.818182,31071.428571,16666.666667,27222.222222,21857.142857,27285.714286,33555.555556,E46NJV
1,22500.0,17142.857143,12500.000000,26000.000000,NaN,14133.333333,16250.000000,10909.090909,10555.555556,10000.000000,15625.000000,20416.666667,12500.000000,0HGSDX
2,NaN,10000.000000,10538.461538,20500.000000,31000.000000,10000.000000,10000.000000,14556.250000,15882.352941,10333.333333,11777.777778,17500.000000,10000.000000,JMQTML
3,30000.0,22000.000000,20557.142857,22728.571429,16000.000000,15545.454545,20582.857143,20200.000000,35240.000000,20000.000000,24000.000000,28500.000000,16530.000000,JKKNTP
4,NaN,NaN,NaN,12666.666667,10000.000000,NaN,14184.615385,18359.523810,26027.777778,46652.941176,40000.000000,NaN,NaN,E4BVRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10050.000000,NaN,NaN,8EEVLJ
49357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23345.000000,NaN,IKPJJJ
49358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.000000,NaN,NaN,NaN,NaN,NaN,4YDCXF
49359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77445.080000,E45DZO
